# sc3nb Test cases

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import sys, time, random, os
import numpy as np

import pythonosc.parsing.osc_types as osc_types
import sc3nb as scn

#from sc3nb import *

In [ ]:
if 'sc' in locals() and sc is not None and isinstance(sc, scn.SC):
    sc.exit()

In [ ]:
sc = scn.startup(start_sclang=False)#scsynth_options=scn.ServerOptions(udp_port=57111)) 

In [ ]:
sc.server.blip()

In [ ]:
sc.server.dump_osc()

In [ ]:
from sc3nb import Bundler

In [ ]:
x = Bundler(0.1, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])

In [ ]:
x.send()

In [ ]:
with Bundler(0.1) as bx:
    bx.add(x)
    bx.wait(0.05)
    bx.add(x)

In [ ]:
with scn.Bundler(0.1) as beep:
    beep.add(scn.Bundler(0.31, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
    beep.add(scn.Bundler(0.41, "/n_free", [-1]))

In [ ]:
with scn.Bundler(0.1) as b:
    b.add(bx)
    b.wait(0.5)
    b.add(bx)
    b.wait(0.5)
    b.add(b)
    b.add(beep)
    b.add(b)

In [ ]:
with scn.Bundler(1) as b:
    syn1 = scn.Synth("s2")
    b.wait(1.0)
    syn2 = scn.Synth("s2", {"freq": 200})
    b.wait(0.5)
    syn1.free()
    b.wait(0.5)
    syn2.free()

In [ ]:
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

In [ ]:
with scn.Bundler(0.2) as b:
    for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
        freq = scn.midicps(60+p)  # see helper fns below
        scn.Synth("s1", {"freq": freq, "dur": 0.5, "num": 1})
        b.wait(0.15)

In [ ]:
with scn.Bundler(0.2) as b:
    for _ in range(15):
        freq = scn.midicps(60+p)  # see helper fns below
        scn.Synth("s1", {"freq": freq, "dur": 0.1, "num": 1})
        b.wait(1)

In [ ]:
for _ in range(15):
    freq = scn.midicps(60+p)  # see helper fns below
    scn.Synth("s1", {"freq": freq, "dur": 0.1, "num": 1})
    time.sleep(1/(100/60))

In [ ]:
b.send()

In [ ]:
isinstance(scn.Bundler(0.11, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]), scn.Bundler)

In [ ]:
with scn.Bundler(0.5) as b:
    b.add(scn.Bundler(0.0, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
    b.add(scn.Bundler(0.5, "/n_free", [-1]))

In [ ]:
with scn.Bundler(0.5) as b:
    b.add(scn.Bundler(0.0, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
    b.wait(0.5)
    b.add(scn.Bundler(0.0, "/n_free", [-1]))

In [ ]:
with scn.Bundler(0.0) as b:
    b.add(scn.Bundler(0.0, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
    b.add(scn.Bundler(0.0, "/n_free", [-1]))

In [ ]:
blip = scn.Bundler()
blip.add(0.11, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip.add(0.31, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2])
blip.add(0.41, "/n_free", [-1])
blip.send()

In [ ]:
with scn.Bundler() as b:
    b.wait(0.6)
    b.add(blip)
    b.wait(0.6)
    b.add(blip)
    b.wait(1.2)
    b.add(blip)
    b.wait(0.6)
    b.add(blip)

In [ ]:
with scn.Bundler(0.1) as b:
    b.add(0.0, "/s_new", ["s2", 10001, 1, 1, "freq", 1000, "amp", 0.05, "num", 2])
    b.add(0.2, "/n_free", [10001])
    b.wait(1.0)
    b.add(blip)
    b.wait(1.0)
    b.add(0.11, "/s_new", ["s1", 10003, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
    b.add(0.31, "/s_new", ["s2", 10004, 1, 1, "freq", 1000, "amp", 0.05, "num", 2])
    b.add(0.41, "/n_free", [10004])

In [ ]:
b.send()

In [ ]:
sc.server.blip()

In [ ]:
sc.server.free_all()

no_sclang_tests

In [ ]:
assert sc.lang == None

In [ ]:
runtime_warning = False
try:
    sc.lang("s;")
except RuntimeWarning:
    runtime_warning = True
assert runtime_warning

In [ ]:
del h

In [ ]:
amp_val = 0.1
h = scn.Synth("s2", {"amp": amp_val})
assert np.allclose(h.amp, amp_val)

In [ ]:
attribute_error = False;
try:
    object.__getattribute__(h, "freq")
except AttributeError:
    attribute_error = True
assert attribute_error

In [ ]:
attribute_error = False;
try:
    h.freq
except AttributeError:
    attribute_error = True
assert attribute_error

In [ ]:
#with self.assertWarns(UserWarning):
h.freq = 200

In [ ]:
h.freq

In [ ]:
h.set("amp", 0.01)

In [ ]:
h.set("freq", 200)

In [ ]:
h.__setattr__("amp", 0.05)

In [ ]:
h.amp

In [ ]:
h.amp = 0.01

In [ ]:
h.freq += 10

In [ ]:
h.amp

In [ ]:
h.free()

In [ ]:
sc.server.free_all()

In [ ]:
sc.server.notify()

In [ ]:
sc.server.msg("/status")

In [ ]:
    #b.add(scn.build_message("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))

In [ ]:
sc.server.init()

In [ ]:
sc.server.msg("/version")

In [ ]:
sc.server.free_all()

In [ ]:
sc.server.dump_osc()

In [ ]:
scn.Synth("s1")

In [ ]:
sc.lang.connect_to_server()

In [ ]:
sc.server.blip()

In [ ]:
sc.server.free_all()

In [ ]:
sc.server.notify()

testing lag

In [ ]:
sc.start_sclang()

In [ ]:
scn.SynthDef.synth_descs.get("s1")

In [ ]:
scn.SynthDef.get_desc("s1")

In [ ]:
scn.Synth("s1")

In [ ]:
num_msg = 200

In [ ]:
scn.SynthDef.synth_descs

In [ ]:
syns = [scn.Synth("s1", dict(dur=0.005), new=False) for _ in range(num_msg)]

In [ ]:
syns[0].new({"dur": 5}) 

In [ ]:
syns[0].freq = 150 

In [ ]:
sc.server.process.read()

In [ ]:
import pythonosc

In [ ]:
bundles = [sc.server.bundle(0).add(synth.new(return_msg=True)).send() for synth in syns]

In [ ]:
latelines = sc.server.process.read()

In [ ]:
lates = np.array(list(map(lambda s: float(s[5:]), latelines.splitlines())))

In [ ]:
lates.min(), lates.max()

In [ ]:
lates.mean(), lates.std()

In [ ]:
sc.server.process.read()

In [ ]:
%sc 200.do({ s.makeBundle(0.0, { Synth(\s1, [\dur, 0.005]); })})

In [ ]:
latelines = sc.server.process.read()

In [ ]:
lates = np.array(list(map(lambda s: float(s[5:]), latelines.splitlines())))

In [ ]:
lates.min(), lates.max()

In [ ]:
lates.mean(), lates.std()

**BUG** bundle nesting

In [ ]:
sc.server.bundle(0, "/s_new", ["s2", 1200, 1, 0, "amp", 0.04]).send()
sc.server.bundle(0.2, "/n_set", [1200, "freq", 200, "num", 1, "amp", 0.2, "pan", 0]).send()
sc.server.bundle(0.4, "/n_free", 1200).send()

In [ ]:
sc.server.process.read()

In [ ]:
t1 = time.time()
blip_bundle = sc.server.bundle(t1)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(sc.server.bundle(t1+0.6, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(sc.server.bundle(t1+0.6+0.5, "/n_free", [-1]))
sc.server.send(blip_bundle.build())

In [ ]:
t1 = time.time() + 0.1
blip_bundle = sc.server.bundle(t1)
blip_bundle.add(sc.server.bundle(t1+0.0, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))
blip_bundle.add(sc.server.bundle(t1+0.6, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(sc.server.bundle(t1+0.6+0.5, "/n_free", [-1]))
sc.server.send(blip_bundle.build())

In [ ]:
sc.server.dump_osc(1)

**BUG** stdin write

In [ ]:
a = sc.lang.process.stdin.write(r"""x = Synth.new(\default, [\freq, 100]);"""+"\n\f")
sc.lang.process.stdin.flush()
b = sc.lang.process.stdin.write(r"""x.free;"""+"\n\f")
a, b

In [ ]:
%sc "Go"
for p in range(1, 3):  # a bouncing ball
    %sc Synth.new(\s1, [\freq, 200+^p])  // this is sc cell so use sc3 comments instead of #

In [ ]:
sc.server.free_all()

Buffer tests

In [ ]:
bufdata = np.random.rand(30000, 1)
buf = Buffer().load_data(bufdata)
np.allclose(buf.to_array(), bufdata)

convert_to_sc_tests

In [ ]:
assert "Array" in sc.lang.cmd("""[1,2,3,4].class""", get_output=True)

In [ ]:
assert "Array" in sc.lang.cmd("""[1,2,3,4].class""", get_output=True)

In [ ]:
python_list = [1,2,3,4]
assert "Array" in sc.lang.cmd("""^python_list.class""", get_output=True)

In [ ]:
np_array = np.array([1,2,3,4])
assert "Array" in sc.lang.cmd("""^np_array.class""", get_output=True)

cmdg tests

In [ ]:
sc_val = sc.lang.cmdg("""1234+23452""")
assert type(sc_val) == int
assert sc_val == 1234+23452

In [ ]:
sc_val = sc.lang.cmdg("""1234.5.squared""")
assert type(sc_val) == float
assert sc_val == 1234.5**2

In [ ]:
sc_val = sc.lang.cmdg('''"soni"++"fication"''')
assert type(sc_val) == str
assert sc_val == "soni"+"fication"

In [ ]:
sc_val = sc.lang.cmdg("""(1,1.1..2)""")
assert type(sc_val) == list
assert np.allclose(sc_val, np.arange(1.0,2.0,0.1)) 

In [ ]:
pylist = [[1]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list
assert sc_val == pylist 

In [ ]:
pylist = [[[1]]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list
assert sc_val == pylist 

In [ ]:
pylist = [[1],[1]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list
assert sc_val == pylist 

In [ ]:
pylist = [[1.0, 1.0],
 [1.100000023841858, 1.100000023841858],
 [1.2000000476837158, 1.2000000476837158],
 [1.2999999523162842, 1.2999999523162842],
 [1.399999976158142, 1.399999976158142],
 [1.5, 1.5],
 [1.600000023841858, 1.600000023841858],
 [1.7000000476837158, 1.7000000476837158],
 [1.7999999523162842, 1.7999999523162842],
 [1.899999976158142, 1.899999976158142]]
sc_val = sc.lang.cmdg(f"""{pylist.__repr__()}""")
assert type(sc_val) == list
assert sc_val == pylist 

In [ ]:
expected_controls = [['out', 'scalar', 0.0],
 ['freq', 'control', 440.0],
 ['amp', 'control', 0.10000000149011612],
 ['pan', 'control', 0.0],
 ['gate', 'control', 1.0]]
controls = sc.lang.cmdg(r"SynthDescLib.global['default'].controls.collect({ arg control; [control.name.asString, control.rate.asString, control.defaultValue]})")
assert controls == expected_controls

Synth param setting/getting

In [ ]:
freq = 42
synth = Synth(name='default', args={"freq": freq, "amp": 0.0})
assert synth.freq == freq
synth.free()

**BUG** nesting bundles

In [ ]:
blip_bundle = bundle_builder(0)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(0.0, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(0.05, "/n_free", [-1]))
sc.server.send(blip_bundle.build())

## Process testing

In [ ]:
sc.server.quit()

In [ ]:
sc.server.boot()

In [ ]:
sc.server.blip()

In [ ]:
sc.server.process.read()

In [ ]:
sc.server.sync()

In [ ]:
sc.server.process.read()

In [ ]:
Synth("s1")

In [ ]:
SC.default.server.free_all()

In [ ]:
Synth("s2")

In [ ]:
sc.server.boot()

In [ ]:
#sc.start_sclang()

In [ ]:
sc.lang.process.popen.poll()

In [ ]:
sc.server.quit()

In [ ]:
%scv s.prepareForRecord;

In [ ]:
sc.server.pid

In [ ]:
sc.lang.kill()

In [ ]:
sc.server.free_all()

In [ ]:
synth_def = SynthDef("klank", r"""
{ |out=0, amp=0.3, freq=440|
    var klank = DynKlank.ar(`[[1,2], [1,1], [1.4,1]], Dust.ar(20), freq);
    Out.ar(out, amp*klank!2);
}""")

In [ ]:
synth_def.add()

In [ ]:
synth = Synth("klank0")

In [ ]:
synth = Synth("s1")

In [ ]:
synth.new()

In [ ]:
synth.freq = 50

In [ ]:
synth.new()

In [ ]:
synth.free()

In [ ]:
synth.new()

In [ ]:
sc.server.boot()

In [ ]:
sc.server.blip()

In [ ]:
sc.server.notify(1)

In [ ]:
sc.server.dump_osc(1)

In [ ]:
sc.lang.cmd("x = Synth(\s2)")

In [ ]:
sc.lang.cmdv("s.defaultGroup")

In [ ]:
sc.lang.cmdg("x.nodeID")

In [ ]:
sc.lang.cmd("x.free")

In [ ]:
sc.server

In [ ]:
sc.server.default_group.new()
sc.server.sync()

In [ ]:
now = time.time()
s2_node_id = sc.server.next_node_id()
target = 1
bundle = bundle_builder(now)

inner_bundle = (bundle_builder(now + 0.7)
                .add_msg("/s_new", ["s1", -1, 1, target, "freq", 400])
                .add_msg("/n_query", [-1])
                .add_msg("/s_new", ["s2", s2_node_id, 1, target, "freq", 100])
                .add_msg("/n_query", [-1])
)
bundle.add_content(inner_bundle.build())

ib2 = (bundle_builder(now + 1.2)
    .add_msg("/n_set", [-1, "freq", 200])
    .add_msg("/s_new", ["s1", -1, 1, target, "freq", 800])
)
bundle.add_content(ib2.build())

bundle.add_content(bundle_builder(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

sc.server.send_bundle(bundle.build())

In [ ]:
sc.server.pid

In [ ]:
t1 = time.time()
blip_bundle = bundle_builder(t1)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(t1+0.6, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.6+0.5, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
print(SC.default)

In [ ]:
%scv "test"

In [ ]:
%scv "test".scramble;

In [ ]:
%sc "test".scramble;

In [ ]:
%scv s.ping

In [ ]:
sc.lang.read()

In [ ]:
%%scv
Routine({(
    Synth.new(\s1, [\freq, 500, \dur, 0.1, \num, 1]);
    0.2.wait;
    x = Synth.new(\s2, [\freq, 1000, \amp, 0.05, \num, 2]);
    0.1.wait;
    x.free;
)}).play

In [ ]:
%scv adaw.

In [ ]:
directory = resources.__file__[:-len("__init__.py")]            
directory            

In [ ]:
sc.server.load_directory(directory)

In [ ]:
# Msgs in bundles dont work

In [ ]:
t1 = time.time() + 0.098
blip_bundle = bundle_builder(t1)
blip_bundle.add(bundle_builder(t1, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))
blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send(blip_bundle.build())

In [ ]:
sc.server.dump_osc()

In [ ]:
t1 = time.time() - 10.098
blip_bundle = bundle_builder(t1)
blip_bundle.add(bundle_builder(t1, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))
blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
blip_bundle = bundle_builder(0.0)
blip_bundle.add(bundle_builder(0.0, "/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))
blip_bundle.add(bundle_builder(0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(0.4, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
bundle = bundle_builder(time.time()+0.1)
bundle.add_content(blip_bundle.build())
sc.server.send_bundle(bundle.build())

In [ ]:
t1 = time.time() + 0.1
blip_bundle = bundle_builder(t1)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send_bundle(bundle_builder(t1).add(blip_bundle.build()).add_msg("/s_new", ["s1", -1, 1, 1]).build())

In [ ]:
t1 = time.time() + 0.1
blip_bundle = bundle_builder(t1)
blip_bundle.add_content(build_message("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1]))
blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
sc.server.free_all()

In [ ]:
blip_bundle.build().dgram

In [ ]:
t1 = time.time()
blip_bundle = bundle_builder(t1)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
blip_bundle.build().dgram

In [ ]:
t1 = time.time()
blip_bundle = bundle_builder(t1)
blip_bundle.add_msg("/s_new", ["s1", -1, 1, 1, "freq", 500, "dur", 0.1, "num", 1])
#blip_bundle.add(bundle_builder(t1+0.3, "/s_new", ["s2", -1, 1, 1, "freq", 1000, "amp", 0.05, "num", 2]))
#blip_bundle.add(bundle_builder(t1+0.4, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
blip_bundle.build().dgram

In [ ]:
sc.server.free_all()

In [ ]:
now = time.time()
s2_node_id = sc.server.next_node_id()
target = 1
bundle = bundle_builder(now)

inner_bundle = (bundle_builder(now + 0.7)
                .add_msg("/s_new", ["s1", -1, 1, target, "freq", 400])
                .add_msg("/n_query", [-1])
                .add_msg("/s_new", ["s2", s2_node_id, 1, target, "freq", 100])
                .add_msg("/n_query", [-1])
)
bundle.add_content(inner_bundle.build())

ib2 = (bundle_builder(now + 1.2)
       .add_msg("/n_set", [-1, "freq", 200])
       .add_msg("/s_new", ["s1", -1, 1, target, "freq", 800])
)
bundle.add_content(ib2.build())

bundle.add_content(bundle_builder(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

sc.server.send_bundle(bundle.build())

In [ ]:
now = 0# time.time()
s2_node_id = sc.server.next_node_id()
target = 1
bundle = bundle_builder(now)

inner_bundle = (bundle_builder(now + 0.7)
                .add_msg("/s_new", ["s1", -1, 1, target, "freq", 400])
                .add_msg("/n_query", [-1])
                .add_msg("/s_new", ["s2", s2_node_id, 1, target, "freq", 100])
                .add_msg("/n_query", [-1])
)
bundle.add_content(inner_bundle.build())

ib2 = (bundle_builder(now + 1.2)
       .add_msg("/n_set", [-1, "freq", 200])
       .add_msg("/s_new", ["s1", -1, 1, target, "freq", 800])
)
bundle.add_content(ib2.build())

bundle.add_content(bundle_builder(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

sc.server.send_bundle(bundle.build())

In [ ]:
sc.server.sync()

In [ ]:
blip_bundle = bundle_builder(0.1, "/s_new", ["default", -1, 1, 0])
blip_bundle.add(bundle_builder(0.2, "/s_new", ["default", -1, 1, 0,]))
blip_bundle.add(bundle_builder(0.1, "/n_free", [-1]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
blip_bundle = bundle_builder(10.1, "/s_new", ["s1", 1041, 1, 0, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(0.2, "/s_new", ["s2", 1042, 1, 0, "freq", 1000, "amp", 0.05, "num", 2]))
blip_bundle.add(bundle_builder(0.1, "/n_free", [1042]))
sc.server.send_bundle(blip_bundle.build())

In [ ]:
sc.server.boot()

In [ ]:
now = time.time()
target = 3
blip_bundle = bundle_builder(now + 0.2, "/s_new", ["default", 1041, 1, target, "freq", 500, "dur", 0.1, "num", 1])
#blip_bundle.add(bundle_builder(0.2, "/s_new", ["s2", 1042, 1, target, "freq", 1000, "amp", 0.05, "num", 2]))
#blip_bundle.add(bundle_builder(now + 1.5, "/n_free", [1041]).build())
sc.server.send_bundle(blip_bundle.build())

In [ ]:
now = time.time()
s2_node_id = sc.server.next_node_id()

bundle = bundle_builder(now)

inner_bundle = bundle_builder(now + 0.7)\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 400])\
    .add_msg("/n_query", [-1])\
    .add_msg("/s_new", ["s2", s2_node_id, 1, 0, "freq", 100])\
    .add_msg("/n_query", [-1])
bundle.add_content(inner_bundle.build())

ib2 = bundle_builder(now + 1.2)\
    .add_msg("/n_set", [-1, "freq", 200])\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 800])
bundle.add_content(ib2.build())

bundle.add_content(bundle_builder(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

sc.server.send_bundle(bundle.build())

In [ ]:
now = time.time()
s2_node_id = sc.server.next_node_id()

bundle = bundle_builder(now)

inner_bundle = bundle_builder(now + 0.7)\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 400])\
    .add_msg("/n_query", [-1])\
    .add_msg("/s_new", ["s2", s2_node_id, 1, 0, "freq", 100])\
    .add_msg("/n_query", [-1])
bundle.add_content(inner_bundle.build())

ib2 = bundle_builder(now + 1.2)\
    .add_msg("/n_set", [-1, "freq", 200])\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 800])
bundle.add_content(ib2.build())

bundle.add_content(bundle_builder(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

sc.server.send_bundle(bundle.build())

In [ ]:
target = 3
blip_bundle = bundle_builder(now + 0.2, "/s_new", ["default", 1041, 1, target, "freq", 500, "dur", 0.1, "num", 1])
blip_bundle.add(bundle_builder(now + 1.5, "/n_free", [1041]).build())
sc.server.send_bundle(blip_bundle.build())

In [ ]:
sc.server.dump_osc(1)

In [ ]:
bb = bundle_builder(1.0, "/s_new", ["s1", 41, 1, 0, "freq", 500, "dur", 0.1, "num", 1])
ib = bundle_builder(2.0, "/s_new", ["s1", 42, 1, 0, "freq", 500, "dur", 0.1, "num", 1])
print(type(ib.build()))
bb.add(ib.build())
bb.add(build_message("/s_new", ["s1", 43, 1, 0, "freq", 600, "dur", 0.2, "num", 2]))
sc.server.send_bundle(bb.build())

In [ ]:
sc.server.default_group.new()

In [ ]:
/g_new [1, 0, 0]

In [ ]:
sc.server.default_group.free()

In [ ]:
sc.server.free_all()

In [ ]:
sc.sclang.server = sc.server

In [ ]:
sc.lang.cmdv("Synth('default')")

In [ ]:
sc.lang.cmdv("s.freeAll")

In [ ]:
sc.server.sync()

In [ ]:
msg = scn.build_message("/g_new", [3, 0, 1])
print(msg.dgram)
sc.server.send_msg(msg)

In [ ]:
sc.server.query_all_nodes()

In [ ]:
sc.server.dump_tree()

In [ ]:
sc.server.free_all()

In [ ]:
sc.sclang.cmdv("Synth('default')")

In [ ]:
sc.sclang.cmdv("s.notify=false")

In [ ]:
sc.server.notify(1)

In [ ]:
sc.server.process.process.poll()

In [ ]:
sc.server.process.process.kill()

In [ ]:
syn = scn.Synth(sc.server)

In [ ]:
syn.sy

In [ ]:
sc.sclang.cmdv('"scramble".scramble')

In [ ]:
sc.server.osc.msg("test")

## sclang 

In [ ]:
sclp = scn.sclang.Sclang()

In [ ]:
sclp.cmd('"scramble".scramble;')
sclp.cmdv('"scramble".scramble;', discard_output=False)

In [ ]:
#sclp.cmdv('s.boot;')

In [ ]:
#print(sclp.read())

In [ ]:
#sclp.cmdv('s.quit;', get_output=True)

In [ ]:
#sclp.read(terminal="exit code")

## SC Server

In [ ]:
scserv = scn.sc_objects.server.SCServer()

In [ ]:
scserv.boot()

In [ ]:
import subprocess

In [ ]:
subprocess.Popen(['C:\\Program Files\\SuperCollider-3.11.0\\scsynth.exe', '-u 57110', '-l 2'])

In [ ]:
subprocess.Popen("C:\\Program Files\\SuperCollider-3.11.0\\scsynth.exe -u 57110 -l 2")

In [ ]:
scserv.remote('127.0.0.1', 57110)

In [ ]:
scserv.status()

In [ ]:
scserv.quit()

In [ ]:
scserv.process.process.poll()

In [ ]:
from sc3nb.osc.osc_communication import build_message

In [ ]:
#scserv.send_msg(build_message("/dumpOSC", 1))

In [ ]:
scserv.osc.get_connection_info()

In [ ]:
scserv.osc.msg("/status")

In [ ]:
scserv.status()

In [ ]:
sclp.osc = scserv.osc

In [ ]:
sclp.cmdg('"scramble".scramble;')

In [ ]:
sclp.server = scserv

In [ ]:
scserv.notify(1)

In [ ]:
sclp.cmd("Synth('s1')")

In [ ]:
sclp.cmdv(r"""s = Server.remote(name: 'remote', addr: NetAddr(^addr, ^port), options: nil , clientID: 0);""", pyvars={"addr": scserv.addr, "port": scserv.server_options.udp_port})

In [ ]:
scn.Synth()

In [ ]:
scserv.process.process.poll()

In [ ]:
d = os.path.dirname(sys.modules['sc3nb'].__file__)
d

In [ ]:
import pkgutil

In [ ]:
import importlib.resources

In [ ]:
try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources


In [ ]:
pkg_resources

In [ ]:
pkg_resources.resource_listdir('sc3nb.ressources', '')

In [ ]:
pkgutil.get_data("sc3nb", "synth")

In [ ]:
"SuperCollider 3 server ready." in scserv.scsynth_process.stdout.peek().decode()

In [ ]:
print(scserv.scsynth_process.stdout.peek().decode())

In [ ]:
scserv.scsynth_process.poll()

In [ ]:
sclp.cmd(r"""s = Server.remote(name: 'remote', addr: NetAddr("127.0.0.1", 57110), options: nil , clientID: 0);""")

## OSC communication

In [ ]:
import threading

In [ ]:
threading.active_count()

In [ ]:
osc = scn.osc.osc_communication.OscCommunication();

In [ ]:
threading.active_count()

In [ ]:
osc.msg_queues

In [ ]:
osc.exit()

In [ ]:
threading.active_count()

## SC Objects

### Node

#### Synth

In [ ]:
sc.server

In [ ]:
s

In [ ]:
synth = sc.Synth("test")

In [ ]:
del synth

In [ ]:
synth = sc.Synth()
time.sleep(0.2)
synth.run(False)

In [ ]:
synth.synth_desc

In [ ]:
synth.synth_desc

In [ ]:
synth

In [ ]:
sc.msg("/s_get", (synth.nodeid, "freq"))

In [ ]:
sc.msg("/s_getn", (synth.nodeid, "freq", 1))

In [ ]:
synth.get("gate")

In [ ]:
synth.fate

In [ ]:
synth.run(True)

In [ ]:
synth.freq = 400

In [ ]:
synth.get("freq")

In [ ]:
synth.new()

In [ ]:
scp.cmdg('"scramble".scramble;', verbose=True)

In [ ]:
scp.osc = osc

In [ ]:
scp.osc = None

In [ ]:
scp.cmdg('"scramble".scramble;')

In [ ]:
scp.cmdg('NetAddr.localAddr;', verbose=True)

In [ ]:
scp.cmdg('NetAddr.langPort;', verbose=True)

## SC

In [ ]:
sc = scn.startup()

In [ ]:
syn = sc.Synth()

In [ ]:
sc.server.free_all()

In [ ]:
sc.Buffer()

In [ ]:
serv1.Buffer()

In [ ]:
sc.server[1].

In [ ]:
sc.osc, sc.sclang

In [ ]:
sc.sclangp.output_queue.queue

In [ ]:
pyvar = r"\\\\default"

In [ ]:
pyvar = r"\\default"

In [ ]:
sc.cmdv("^pyvar")
sc.cmdv("^pyvar.class")

In [ ]:
syn_def = sc.SynthDef("random", """{ |out, l_freq=400 |
    Out.ar(out,
        SinOsc.ar(rrand(400, 800), 0, 0.2) * Line.kr(1, 0, 1, doneAction: Done.freeSelf)
    )
}""")

In [ ]:
syn_name = syn_def.add()

In [ ]:
syn = sc.Synth(syn_name)

In [ ]:
syn.synth_args

In [ ]:
print(sc.msg("/s_get", [syn.nodeid, "pan"], sync=False))

In [ ]:
sc.osc.msg_queues

In [ ]:
syn.pan = 1

In [ ]:
syn.pan = -1 * syn.pan

In [ ]:
syn.free()

In [ ]:
ip, port = sc.get_connection_info()[0]

In [ ]:
%scv r

In [ ]:
sc.osc.msg_queues

In [ ]:
sc.free_all()

In [ ]:
sc.cmd(r"OSCFunc({|msg, time, addr, recvPort| r['scn'].sendMsg(msg); }, '/done', s.addr);")

In [ ]:
sc.cmd(r"""r['sc3nb'].sendMsg(*["/hello", "there"]);""")

In [ ]:
%scv r